<a href="https://colab.research.google.com/github/christinium/Health/blob/main/Echo_Fine_Tunning_vs_Prompt_Stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Echocardiogram Analysis: Comparing Fine-tuned vs Prompt-only Models

## Purpose
This notebook compares two approaches for extracting structured labels from
echocardiogram reports using the Gemma language model:

1. **Fine-tuned Model**: Gemma model fine-tuned on labeled echo reports
2. **Prompt-only Model**: Base Gemma-2B-it model using detailed prompting

## Task
Both models extract 19 cardiac features from unstructured echo reports and
classify each feature using a standardized coding schema (-3 to 3, representing
conditions from "inadequate to evaluate" to "severe dysfunction").

## Evaluation Method: Strict Accuracy
This notebook uses **Method 2 (Strict)** evaluation, where:
- Failed predictions (unparseable outputs) count as **wrong**
- All predictions are evaluated, not just successful parses
- This provides a fair comparison reflecting real-world deployment scenarios

## Key Findings

### Fine-tuned Model Performance
- **Average Per-Label Accuracy**: ~99.8%
- **Exact Match Accuracy**: ~98.1% (all 19 labels correct)
- **Failed Predictions**: <0.1%
- The fine-tuned model consistently produces valid, structured output

### Prompt-only Model Performance  
- **Average Per-Label Accuracy**: ~15-20%
- **Exact Match Accuracy**: <1%
- **Failed Predictions**: ~50%
- The prompt-only model frequently fails to produce parseable output despite
  detailed instructions and medical context

### Impact of Fine-tuning
Fine-tuning improved accuracy by **~80 percentage points** and reduced failed
predictions from 50% to nearly 0%. This demonstrates that for structured
extraction tasks from specialized medical text, fine-tuning is essential—prompt
engineering alone is insufficient.

## Files Required
- `test_inference_results_batch_run.csv` - Fine-tuned model predictions
- `test_df_with_gemma_predictions.csv` - Prompt-only model predictions

Both files should contain ground truth labels for comparison.

In [1]:
import pandas as pd
import numpy as np
import ast
import re
from datetime import datetime


In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set up working directory
import os
WORKING_DIR = '/content/drive/MyDrive/echo_training/'  # Change this to your preferred location
os.makedirs(WORKING_DIR, exist_ok=True)
os.chdir(WORKING_DIR)

Mounted at /content/drive


In [3]:

# ==============================================================================
# CONFIGURATION
# ==============================================================================

# Label definitions
LABEL_NAMES = [
    'LA_cavity', 'RA_dilated', 'LV_systolic', 'LV_cavity',
    'LV_wall', 'RV_cavity', 'RV_systolic', 'AV_stenosis',
    'MV_stenosis', 'TV_regurgitation', 'TV_stenosis',
    'TV_pulm_htn', 'AV_regurgitation', 'MV_regurgitation',
    'RA_pressure', 'LV_diastolic', 'RV_volume_overload',
    'RV_wall', 'RV_pressure_overload'
]


In [4]:
# ==============================================================================
# FUNCTIONS
# ==============================================================================

def load_predictions(model_type):
    """Load predictions for specified model type."""
    if model_type == "fine-tuned":
        df = pd.read_csv('test_inference_results_batch_run.csv')
        return df, 'prediction_text', 'true_labels'
    else:
        df = pd.read_csv('test_df_with_gemma_predictions.csv')
        df = df.rename(columns={df.columns[0]: 'id_num'})
        return df, 'gemma_result', 'labels'

In [5]:
def parse_prediction(pred_text: str, model_type: str) -> list:
    """Parse model output into label list."""
    if model_type == "fine-tuned":
        predicted = []
        lines = str(pred_text).split('\n')

        for label_name in LABEL_NAMES:
            found = False
            for line in lines:
                if label_name in line and ':' in line:
                    try:
                        value_str = line.split(':')[1].strip()
                        value = int(value_str)
                        predicted.append(value)
                        found = True
                        break
                    except:
                        pass
            if not found:
                predicted.append(None)
        return predicted
    else:
        pattern = r'\*?\*?([A-Z_a-z]+)\s*:\s*(\d+)\*?\*?'
        matches = re.findall(pattern, str(pred_text))
        parsed_data = {}
        for key, value in matches:
            if key in LABEL_NAMES:
                parsed_data[key] = int(value)
        return [parsed_data.get(label, None) for label in LABEL_NAMES]

In [6]:
def calculate_accuracy(df):
    """Calculate accuracy treating unparseable predictions as WRONG."""
    accuracy_results = []

    for i, label_name in enumerate(LABEL_NAMES):
        true_vals = df['labels_parsed'].apply(
            lambda x: x[i] if isinstance(x, list) and i < len(x) else None
        ).values

        pred_vals = df['pred_labels'].apply(
            lambda x: x[i] if isinstance(x, list) and i < len(x) else None
        ).values

        valid_true_mask = ~pd.isna(true_vals)
        true_vals_all = true_vals[valid_true_mask]
        pred_vals_all = pred_vals[valid_true_mask]

        correct = sum(1 for tv, pv in zip(true_vals_all, pred_vals_all)
                     if pd.notna(pv) and tv == pv)

        total = len(true_vals_all)
        accuracy = correct / total if total > 0 else 0
        failed = sum(pd.isna(pred_vals_all))

        accuracy_results.append({
            'label': label_name,
            'correct': correct,
            'total': total,
            'accuracy': accuracy,
            'failed': failed
        })

    exact_matches = sum(
        1 for idx in range(len(df))
        if (df.iloc[idx]['pred_labels'] is not None and
            df.iloc[idx]['labels_parsed'] == df.iloc[idx]['pred_labels'])
    )

    return accuracy_results, exact_matches

In [7]:
def display_results(model_type, accuracy_results, exact_matches, total_samples):
    """Display results for a model."""
    print("\n" + "="*80)
    print(f"{model_type.upper()} MODEL RESULTS")
    print("="*80)

    for result in accuracy_results:
        failed_str = f" ({result['failed']} failed)" if result['failed'] > 0 else ""
        print(f"{result['label']:20s}: {result['correct']:5d}/{result['total']:5d} = {result['accuracy']:.4f}{failed_str}")

    avg_accuracy = np.mean([r['accuracy'] for r in accuracy_results])
    total_failed = sum(r['failed'] for r in accuracy_results)
    total_predictions = total_samples * len(LABEL_NAMES)

    print("\n" + "-"*80)
    print(f"Average Per-Label Accuracy: {avg_accuracy:.4f} ({avg_accuracy*100:.2f}%)")
    print(f"Exact Match Accuracy: {exact_matches}/{total_samples} = {exact_matches/total_samples:.4f}")
    print(f"Total Failed Predictions: {total_failed}/{total_predictions} ({total_failed/total_predictions*100:.2f}%)")
    print("="*80)


In [8]:


# ==============================================================================
# EVALUATE BOTH MODELS
# ==============================================================================

results_comparison = {}

for model_type in ["fine-tuned", "prompt-only"]:
    print(f"\n{'='*80}")
    print(f"Processing {model_type.upper()} model...")
    print('='*80)

    # Load data
    df, pred_col, label_col = load_predictions(model_type)
    print(f"Loaded {len(df)} predictions")

    # Parse predictions
    print("Parsing predictions...")
    df['pred_labels'] = df[pred_col].apply(lambda x: parse_prediction(x, model_type))
    df['labels_parsed'] = df[label_col].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else x
    )

    # Calculate accuracy
    print("Calculating accuracy...")
    accuracy_results, exact_matches = calculate_accuracy(df)

    # Store results
    results_comparison[model_type] = {
        'accuracy_results': accuracy_results,
        'exact_matches': exact_matches,
        'total_samples': len(df)
    }

    # Display results
    display_results(model_type, accuracy_results, exact_matches, len(df))

    # Save individual results
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    results_df = pd.DataFrame(accuracy_results)
    results_df.to_csv(f'accuracy_{model_type}_strict_{timestamp}.csv', index=False)
    print(f"\nSaved to: accuracy_{model_type}_strict_{timestamp}.csv")




Processing FINE-TUNED model...
Loaded 6608 predictions
Parsing predictions...
Calculating accuracy...

FINE-TUNED MODEL RESULTS
LA_cavity           :  6596/ 6608 = 0.9982 (1 failed)
RA_dilated          :  6607/ 6608 = 0.9998 (1 failed)
LV_systolic         :  6598/ 6608 = 0.9985 (1 failed)
LV_cavity           :  6604/ 6608 = 0.9994 (1 failed)
LV_wall             :  6597/ 6608 = 0.9983 (1 failed)
RV_cavity           :  6605/ 6608 = 0.9995 (1 failed)
RV_systolic         :  6601/ 6608 = 0.9989 (1 failed)
AV_stenosis         :  6599/ 6608 = 0.9986 (1 failed)
MV_stenosis         :  6602/ 6608 = 0.9991 (1 failed)
TV_regurgitation    :  6602/ 6608 = 0.9991 (1 failed)
TV_stenosis         :  6606/ 6608 = 0.9997 (1 failed)
TV_pulm_htn         :  6603/ 6608 = 0.9992 (1 failed)
AV_regurgitation    :  6579/ 6608 = 0.9956 (1 failed)
MV_regurgitation    :  6581/ 6608 = 0.9959 (1 failed)
RA_pressure         :  6607/ 6608 = 0.9998 (1 failed)
LV_diastolic        :  6598/ 6608 = 0.9985 (1 failed)
RV_volu

In [10]:
# ==============================================================================
# SIDE-BY-SIDE COMPARISON
# ==============================================================================

print("\n\n" + "="*80)
print("SIDE-BY-SIDE COMPARISON")
print("="*80)
print(f"\n{'Label':<20} {'Fine-tuned':>15} {'Prompt-only':>15} {'Difference':>15}")
print("-"*80)

for i, label in enumerate(LABEL_NAMES):
    ft_acc = results_comparison['fine-tuned']['accuracy_results'][i]['accuracy']
    po_acc = results_comparison['prompt-only']['accuracy_results'][i]['accuracy']
    diff = ft_acc - po_acc

    print(f"{label:<20} {ft_acc:>14.4f} {po_acc:>14.4f} {diff:>+14.4f}")

print("\n" + "-"*80)
ft_avg = np.mean([r['accuracy'] for r in results_comparison['fine-tuned']['accuracy_results']])
po_avg = np.mean([r['accuracy'] for r in results_comparison['prompt-only']['accuracy_results']])
ft_exact = results_comparison['fine-tuned']['exact_matches'] / results_comparison['fine-tuned']['total_samples']
po_exact = results_comparison['prompt-only']['exact_matches'] / results_comparison['prompt-only']['total_samples']

print(f"{'Average Accuracy':<20} {ft_avg:>14.4f} {po_avg:>14.4f} {ft_avg-po_avg:>+14.4f}")
print(f"{'Exact Match':<20} {ft_exact:>14.4f} {po_exact:>14.4f} {ft_exact-po_exact:>+14.4f}")
print("="*80)



SIDE-BY-SIDE COMPARISON

Label                     Fine-tuned     Prompt-only      Difference
--------------------------------------------------------------------------------
LA_cavity                    0.9982         0.0802        +0.9180
RA_dilated                   0.9998         0.0947        +0.9051
LV_systolic                  0.9985         0.0322        +0.9663
LV_cavity                    0.9994         0.0699        +0.9295
LV_wall                      0.9983         0.1270        +0.8714
RV_cavity                    0.9995         0.1094        +0.8901
RV_systolic                  0.9989         0.1265        +0.8724
AV_stenosis                  0.9986         0.0454        +0.9532
MV_stenosis                  0.9991         0.0944        +0.9047
TV_regurgitation             0.9991         0.0988        +0.9003
TV_stenosis                  0.9997         0.1206        +0.8791
TV_pulm_htn                  0.9992         0.1203        +0.8789
AV_regurgitation             0.